In [93]:
import pandas as pd

In [94]:
train = pd.read_csv('Train_Stations.csv')

In [95]:
train.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location
0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,False,False,False,False,False,False,False,True,False,"(41.85182, -87.745336)"
1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,False,False,False,False,False,False,False,True,False,"(41.853839, -87.714842)"
2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,False,False,True,False,False,False,False,False,False,"(41.778943, -87.644244)"
3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,False,True,False,False,False,False,False,False,False,"(41.984246, -87.838028)"
4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,False,True,False,False,False,False,False,False,False,"(41.87592, -87.659458)"


In [96]:
train.Location = train.Location.apply(lambda x:x.replace('(',''))
train.Location = train.Location.apply(lambda x:x.replace(')',''))
temp = train.Location.str.split(',',expand = True)

In [97]:
train['Lat'] = temp[0]
train['Long'] = temp[1]

In [98]:
train.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Lat,Long
0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,False,False,False,False,False,False,False,True,False,"41.85182, -87.745336",41.85182,-87.745336
1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,False,False,False,False,False,False,False,True,False,"41.853839, -87.714842",41.853839,-87.714842
2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,False,False,True,False,False,False,False,False,False,"41.778943, -87.644244",41.778943,-87.644244
3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,False,True,False,False,False,False,False,False,False,"41.984246, -87.838028",41.984246,-87.838028
4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,False,True,False,False,False,False,False,False,False,"41.87592, -87.659458",41.87592,-87.659458


In [99]:
train['Lat'] = train.Lat.apply(pd.to_numeric)
train['Long'] = train.Long.apply(pd.to_numeric)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STOP_ID                   300 non-null    int64  
 1   DIRECTION_ID              300 non-null    object 
 2   STOP_NAME                 300 non-null    object 
 3   STATION_NAME              300 non-null    object 
 4   STATION_DESCRIPTIVE_NAME  300 non-null    object 
 5   MAP_ID                    300 non-null    int64  
 6   ADA                       300 non-null    bool   
 7   RED                       300 non-null    bool   
 8   BLUE                      300 non-null    bool   
 9   G                         300 non-null    bool   
 10  BRN                       300 non-null    bool   
 11  P                         300 non-null    bool   
 12  Pexp                      300 non-null    bool   
 13  Y                         300 non-null    bool   
 14  Pnk       

In [100]:
!pip install pyshp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
from shapely.geometry import shape, Point
import shapefile

In [102]:
areas = pd.read_csv('CommAreas.csv')
comm = areas.AREA_NUMBE

In [103]:
r = shapefile.Reader("geo_export_857e7ab3-ee77-40cd-b5e0-5222e9567269.shp")

# get the shapes
shapes = r.shapes()

# build a shapely polygon from your shape
polygon = []
for x in range(0,len(r)):
  polygon.append(shape(shapes[x])) 

list = []
for y in train.index:
  flag = 0
  for x in range(0,len(r)):
    if polygon[x].contains(Point(float(train.Long[y]),float(train.Lat[y]))):
      list.append(x + 1)
      flag = 1
  if flag == 0:
    list.append(0)

len(list)

300

In [104]:
train['Comm_Area'] = list

In [105]:
train.head()

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,Lat,Long,Comm_Area
0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,False,False,False,False,False,False,False,True,False,"41.85182, -87.745336",41.851820,-87.745336,0
1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,False,False,False,False,False,False,False,True,False,"41.853839, -87.714842",41.853839,-87.714842,30
2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,False,False,True,False,False,False,False,False,False,"41.778943, -87.644244",41.778943,-87.644244,66
3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,False,True,False,False,False,False,False,False,False,"41.984246, -87.838028",41.984246,-87.838028,75
4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,False,True,False,False,False,False,False,False,False,"41.87592, -87.659458",41.875920,-87.659458,29


In [106]:
train.to_csv('Train_Preprocessed.csv')

In [107]:
hospital = pd.read_csv('Hospitals_Processed.csv')

In [108]:
hospital.head()

,Unnamed: 0,Site Name,Clinic Type,Site Number,Hours of Operation,Street Address,City,State,ZIP,Phone 1,...,Fax,Public Health Nursing,Family Case Management,Healthy Start Program,Healthy Families Program,WIC,Latitude,Longitude,Location,Comm_Area
0,0,Austin STI Specialty Clinic,STI Specialty Clinic,NaN,Mon and Wed: 8:00 a.m. - 4:00 p.m.; Tues: 9:00...,"4909 W. Division St., Suite 411",Chicago,IL,60651,312-746-4871,...,312-746-4637,NaN,NaN,NaN,NaN,NaN,41.902212,-87.748845,"(41.902212, -87.748845)",25
1,1,Englewood MHC,Mental Health Clinic,NaN,Mon - Fri: 8:30 a.m. – 4:30 p.m.,641 W. 63rd St,Chicago,IL,60621,312-747-7496,...,NaN,NaN,NaN,NaN,NaN,NaN,41.779692,-87.641428,"(41.779692, -87.641428)",68
2,2,Englewood STI Specialty Clinic,STI Specialty Clinic,NaN,"Mon, Wed & Fri: 8:00 a.m. - 4:00 p.m.; Tues & ...","641 W. 63rd St., Lower Level",Chicago,IL,60621,312-747-8900,...,312-747-0292,NaN,NaN,NaN,NaN,NaN,41.779692,-87.641428,"(41.779692, -87.641428)",68
3,3,Greater Grand/MID-South MHC,Mental Health Clinic,NaN,Mon - Fri: 8:30 a.m. – 4:30 p.m.,4314 S. Cottage Grove,Chicago,IL,60653,312-747-0036,...,NaN,NaN,NaN,NaN,NaN,NaN,41.816326,-87.606847,"(41.816326, -87.606847)",38
4,4,Alivio Medical Center,WIC Clinic,106.0,"Mon, Tues, Thurs & Fri: 8:00 a.m. - 4:30 p.m.;...",2355 S. Western Ave.,Chicago,IL,60608,773-843-4220,...,773-650-1232,NaN,NaN,NaN,NaN,Y,41.848583,-87.685394,"(41.848583, -87.685394)",31


In [109]:
hospital_comm = hospital.groupby('Comm_Area')['Unnamed: 0'].count().reset_index()


In [110]:
hospital_comm
  

,Comm_Area,Unnamed: 0
0,2,1
1,3,1
2,6,1
3,13,1
4,24,2
5,25,2
6,28,1
7,30,1
8,31,3
9,32,1


In [ ]:
comm_area = pd.read_csv('Comm_area_aggregations.csv')

In [141]:
hospital_comm = hospital_comm.rename(columns={'Comm_Area':'COMMAREA','BusCount':'HosCount'})
comm_area = comm_area.merge(hospital_comm, on="COMMAREA", how='left')
comm_area.head(74)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,HosCount_x,SchoolCount,TrainCount,BusCount,HosCount_y
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0.0,7,2.0,7,NaN
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1.0,10,NaN,2,1.0
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1.0,7,4.0,3,1.0
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0.0,6,8.0,5,NaN
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0.0,6,NaN,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,72,72,73,553,86,24644,0,12,199,410,25218,0.0,11,NaN,2,NaN
64,73,73,74,16972,208,894,0,21,261,1414,18512,0.0,4,NaN,2,NaN
65,74,74,75,10270,355,15609,0,33,403,2126,26956,0.0,7,4.0,1,NaN
66,75,75,76,18471,3125,2390,0,42,364,3785,26313,0.0,1,8.0,1,NaN


In [131]:
school = pd.read_csv('Schools_Preprocessed.csv')
school.head()

,Unnamed: 0,the_geom,School_ID,Short_Name,Address,Grade_Cat,Lat,Long,Comm_Area
0,0,POINT (-87.6266211792559 41.81924917367548),610590,BRONZEVILLE CLASSICAL,8 W ROOT ST,ES,41.819249,-87.626621,38
1,1,POINT (-87.64556149971644 41.907796795867235),400113,CICS - CHICAGOQUEST HS,1443 N OGDEN AVE,HS,41.907797,-87.645561,8
2,2,POINT (-87.55990690211836 41.74267138259229),400168,GREAT LAKES,8401 S SAGINAW AVE,ES,41.742671,-87.559907,46
3,3,POINT (-87.63756486350432 41.91389953306062),610033,LASALLE,1734 N ORLEANS ST,ES,41.913900,-87.637565,7
4,4,POINT (-87.73599882158203 41.92791840313385),609716,KELVYN PARK HS,4343 W WRIGHTWOOD AVE,HS,41.927918,-87.735999,20


In [132]:
school_comm = school.groupby('Comm_Area')['Unnamed: 0'].count().reset_index()
school_comm

,Comm_Area,Unnamed: 0
0,1,7
1,2,10
2,3,7
3,4,6
4,5,6
...,...,...
72,73,11
73,74,4
74,75,7
75,76,1


In [142]:
school_comm = school_comm.rename(columns={'Comm_Area':'COMMAREA','Unnamed: 0':'SchoolCount'})
comm_area = comm_area.merge(school_comm, on="COMMAREA", how='left')
comm_area.head(74)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,HosCount_x,SchoolCount_x,TrainCount,BusCount,HosCount_y,SchoolCount_y
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0.0,7,2.0,7,NaN,7
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1.0,10,NaN,2,1.0,10
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1.0,7,4.0,3,1.0,7
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0.0,6,8.0,5,NaN,6
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0.0,6,NaN,5,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,72,72,73,553,86,24644,0,12,199,410,25218,0.0,11,NaN,2,NaN,11
64,73,73,74,16972,208,894,0,21,261,1414,18512,0.0,4,NaN,2,NaN,4
65,74,74,75,10270,355,15609,0,33,403,2126,26956,0.0,7,4.0,1,NaN,7
66,75,75,76,18471,3125,2390,0,42,364,3785,26313,0.0,1,8.0,1,NaN,1


In [51]:
comm_area.head(50)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,Hospitals,Schools,Trains,Buses
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0,7,10,10
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1,10,0,0
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1,7,8,8
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0,6,8,8
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0,6,4,4
5,5,5,6,102803,10009,5694,0,185,1486,7831,118598,1,10,18,18
6,6,6,7,74565,6706,3810,0,85,843,4385,84802,0,7,6,6
7,7,7,8,86791,14270,9942,0,153,905,5947,110687,0,8,14,14
8,8,8,9,10708,441,178,0,6,151,972,11531,0,3,0,0
9,9,9,10,38026,3100,804,0,57,756,5072,43604,0,8,2,2


In [52]:
trains = pd.read_csv('Train_Preprocessed.csv')
trains.head()

,Unnamed: 0,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,...,BRN,P,Pexp,Y,Pnk,O,Location,Lat,Long,Comm_Area
0,0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,False,False,...,False,False,False,False,True,False,"41.85182, -87.745336",41.851820,-87.745336,0
1,1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,False,False,...,False,False,False,False,True,False,"41.853839, -87.714842",41.853839,-87.714842,30
2,2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,False,False,...,False,False,False,False,False,False,"41.778943, -87.644244",41.778943,-87.644244,66
3,3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,False,True,...,False,False,False,False,False,False,"41.984246, -87.838028",41.984246,-87.838028,75
4,4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,False,True,...,False,False,False,False,False,False,"41.87592, -87.659458",41.875920,-87.659458,29


In [53]:
train_comm = trains.groupby('Comm_Area')['Unnamed: 0'].count().reset_index()
train_comm

,Comm_Area,Unnamed: 0
0,0,40
1,1,2
2,3,4
3,4,8
4,6,8
5,7,2
6,9,4
7,10,10
8,11,2
9,14,6


In [134]:
train_comm = train_comm.rename(columns={'Comm_Area':'COMMAREA','Unnamed: 0':'TrainCount'})
comm_area = comm_area.merge(train_comm, on="COMMAREA", how='left')
comm_area.head(20)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,HosCount,SchoolCount,TrainCount
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0.0,7,2.0
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1.0,10,NaN
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1.0,7,4.0
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0.0,6,8.0
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0.0,6,NaN
5,5,5,6,102803,10009,5694,0,185,1486,7831,118598,1.0,10,8.0
6,6,6,7,74565,6706,3810,0,85,843,4385,84802,0.0,7,2.0
7,7,7,8,86791,14270,9942,0,153,905,5947,110687,0.0,8,NaN
8,9,9,10,38026,3100,804,0,57,756,5072,43604,0.0,8,10.0
9,10,10,11,19498,3258,805,0,39,580,5044,25981,0.0,2,2.0


In [56]:
comm_area.head(20)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,Hospitals,Schools,Trains,Buses
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0,7,2,10
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1,10,0,0
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1,7,4,8
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0,6,8,8
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0,6,0,4
5,5,5,6,102803,10009,5694,0,185,1486,7831,118598,1,10,8,18
6,6,6,7,74565,6706,3810,0,85,843,4385,84802,0,7,2,6
7,7,7,8,86791,14270,9942,0,153,905,5947,110687,0,8,0,14
8,8,8,9,10708,441,178,0,6,151,972,11531,0,3,4,0
9,9,9,10,38026,3100,804,0,57,756,5072,43604,0,8,10,2


In [57]:
bus = pd.read_csv('Bus_Preprocessed.csv')
bus.head()

,Unnamed: 0,Stop_ID,CTA Stop Name,Direction,Routes,Ward,LONGITUDE,LATITUDE,Location,Phase,COMM_AREA
0,0,68,Jackson & Franklin,EB,"1,7,28,126,130,151,156",42,-87.635401,41.878051,"(41.878051, -87.635401)",2,32
1,1,69,Jackson & Financial Place,EB,"1,7,28,126,130,132,151",42,-87.632868,41.878081,"(41.878081, -87.632868)",1,32
2,2,73,Michigan & Van Buren/Congress,SB,"1,3,4,7,J14,26,28,126,132,143,147",42,-87.624396,41.876271,"(41.876271, -87.624396)",1,32
3,3,75,Michigan & Van Buren,NB,"1,7,28,126",42,-87.624160,41.876690,"(41.87669, -87.62416)",2,32
4,4,76,Michigan & Jackson,NB,"3,4,6,J14,26,143",42,-87.624193,41.878135,"(41.878135, -87.624193)",1,32


In [64]:
bus_comm = bus.groupby('COMM_AREA')['Unnamed: 0'].count().reset_index()
bus_comm.head(20)

,COMM_AREA,Unnamed: 0
0,1,7
1,2,2
2,3,3
3,4,5
4,5,5
5,6,20
6,7,9
7,8,26
8,10,2
9,11,1


In [135]:
bus_comm = bus_comm.rename(columns={'COMM_AREA':'COMMAREA','Unnamed: 0':'BusCount'})
comm_area = comm_area.merge(bus_comm, on="COMMAREA", how='left')
comm_area.head(20)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,HosCount,SchoolCount,TrainCount,BusCount
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0.0,7,2.0,7
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1.0,10,NaN,2
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1.0,7,4.0,3
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0.0,6,8.0,5
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0.0,6,NaN,5
5,5,5,6,102803,10009,5694,0,185,1486,7831,118598,1.0,10,8.0,20
6,6,6,7,74565,6706,3810,0,85,843,4385,84802,0.0,7,2.0,9
7,7,7,8,86791,14270,9942,0,153,905,5947,110687,0.0,8,NaN,26
8,9,9,10,38026,3100,804,0,57,756,5072,43604,0.0,8,10.0,2
9,10,10,11,19498,3258,805,0,39,580,5044,25981,0.0,2,2.0,1


In [61]:
comm_area.head(20)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,Hospitals,Schools,Trains,Buses
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0,7,2,2
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1,10,0,0
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1,7,4,4
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0,6,8,8
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0,6,0,0
5,5,5,6,102803,10009,5694,0,185,1486,7831,118598,1,10,8,8
6,6,6,7,74565,6706,3810,0,85,843,4385,84802,0,7,2,2
7,7,7,8,86791,14270,9942,0,153,905,5947,110687,0,8,0,0
8,8,8,9,10708,441,178,0,6,151,972,11531,0,3,4,4
9,9,9,10,38026,3100,804,0,57,756,5072,43604,0,8,10,10


In [136]:
comm_area.fillna(0)

,Unnamed: 0,Unnamed: 0.1,COMMAREA,White,Asian,Black,Hispanic,Pacific Islander,Native,Other,Population,HosCount,SchoolCount,TrainCount,BusCount
0,0,0,1,155002,6369,22824,0,265,4592,19392,192610,0.0,7,2.0,7
1,1,1,2,93661,22021,16107,0,205,3003,16068,139958,1.0,10,0.0,2
2,2,2,3,44316,7174,12409,0,90,1350,6646,66050,1.0,7,4.0,3
3,3,3,4,57326,5674,2577,0,121,1493,6409,67635,0.0,6,8.0,5
4,4,4,5,63432,3717,4475,0,136,1310,4659,71808,0.0,6,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,72,72,73,553,86,24644,0,12,199,410,25218,0.0,11,0.0,2
64,73,73,74,16972,208,894,0,21,261,1414,18512,0.0,4,0.0,2
65,74,74,75,10270,355,15609,0,33,403,2126,26956,0.0,7,4.0,1
66,75,75,76,18471,3125,2390,0,42,364,3785,26313,0.0,1,8.0,1


In [137]:
comm_area.to_csv('Comm_area_aggregations.csv')